In [1]:
import xarray
import numpy as np
import pandas
import h5py
import datetime
from ulmo.utils import image_utils
from ulmo.plotting import plotting
import matplotlib.pyplot as plt
import seaborn as sns
import os

/Users/rachelstumpf/ulmo/ulmo/utils/image_utils.py:8: UserWarning: healpy not installed.  Not all functions will work!
  warnings.warn("healpy not installed.  Not all functions will work!")


In [2]:
# check extraction tables

table_2012 = pandas.read_parquet('/Users/rachelstumpf/Desktop/parquet/SST_L3S_VIIRS_2012.parquet')
table_2013 = pandas.read_parquet('/Users/rachelstumpf/Desktop/parquet/SST_L3S_VIIRS_2013.parquet')
table_2014 = pandas.read_parquet('/Users/rachelstumpf/Desktop/parquet/SST_L3S_VIIRS_2014.parquet')
table_2015 = pandas.read_parquet('/Users/rachelstumpf/Desktop/parquet/SST_L3S_VIIRS_2015.parquet')
table_2016 = pandas.read_parquet('/Users/rachelstumpf/Desktop/parquet/SST_L3S_VIIRS_2016.parquet')
table_2017 = pandas.read_parquet('/Users/rachelstumpf/Desktop/parquet/SST_L3S_VIIRS_2017.parquet')
table_2018 = pandas.read_parquet('/Users/rachelstumpf/Desktop/parquet/SST_L3S_VIIRS_2018.parquet')
table_2019 = pandas.read_parquet('/Users/rachelstumpf/Desktop/parquet/SST_L3S_VIIRS_2019.parquet')
table_2020 = pandas.read_parquet('/Users/rachelstumpf/Desktop/parquet/SST_L3S_VIIRS_2020.parquet')

table_all = pandas.concat([table_2012, table_2013, table_2014, table_2015,
                        table_2016, table_2017, table_2018, table_2019, table_2020])
table_all.reset_index(drop=True, inplace=True)
table_all = table_all.dropna(subset=['LL'])
table_all.loc[:,'LL_diff'] = table_all.loc[:,'VIIRS_LL'] - table_all.loc[:,'LL']

In [3]:
table_all[['row', 'col']].head(100)

,row,col
0,3501,123
1,3462,152
2,3417,209
3,3424,177
4,3431,142
...,...,...
97,6504,6131
98,6511,6175
99,6518,6219
100,6518,6025


In [4]:
top_frequent_combinations = 100

combination_counts = table_all.groupby(['row', 'col']).size().reset_index(name='count')
top_combinations = combination_counts.sort_values(by='count', ascending=False).head(top_frequent_combinations)
result = pandas.merge(top_combinations, table_all, on=['row', 'col'])

result

,row,col,count,VIIRS_row,VIIRS_col,VIIRS_UID,VIIRS_LL,VIIRS_pp_file,VIIRS_pp_idx,VIIRS_T90,...,pp_type,mean_temperature,Tmin,Tmax,T90,T10,clear_fraction,DT,LL,LL_diff
0,5430,14681,10,2928,960,134370747929426440,443.155518,s3://viirs/PreProc/VIIRS_2012_95clear_192x192_...,699476,24.579987,...,0,24.346169,23.790009,24.750000,24.470001,24.220001,1.0,0.250000,617.902161,-174.746643
1,5430,14681,10,3696,2400,134470743329426359,406.137939,s3://viirs/PreProc/VIIRS_2012_95clear_192x192_...,308836,24.809998,...,0,24.443045,23.980011,25.109985,24.679993,24.160004,1.0,0.519989,481.129028,-74.991089
2,5430,14681,10,144,2592,135170740429427225,492.862732,s3://viirs/PreProc/VIIRS_2012_95clear_192x192_...,1560,26.100006,...,0,25.405518,24.589996,25.989990,25.829987,24.940002,1.0,0.889984,521.931396,-29.068665
3,5430,14681,10,4848,1920,137770747529426775,278.187134,s3://viirs/PreProc/VIIRS_2013_95clear_192x192_...,204024,25.309998,...,0,24.825085,24.190002,25.459991,25.339996,24.459991,1.0,0.880005,366.600220,-88.413086
4,5430,14681,10,1584,1536,140370745529426797,603.196899,s3://viirs/PreProc/VIIRS_2014_95clear_192x192_...,741685,25.950012,...,0,25.977936,25.730011,26.440002,26.100006,25.869995,1.0,0.230011,643.774719,-40.577820
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
730,5429,14783,7,4272,192,150470763729631107,-146.843933,s3://viirs/PreProc/VIIRS_2017_95clear_192x192_...,416038,27.359985,...,0,25.656935,24.489990,26.359985,25.980011,25.209991,1.0,0.770020,264.559814,-411.403748
731,5429,14783,7,2832,2304,153670769529631416,159.442505,s3://viirs/PreProc/VIIRS_2018_95clear_192x192_...,989523,24.670013,...,0,24.292171,23.220001,25.059998,24.559998,23.959991,1.0,0.600006,290.809937,-131.367432
732,5429,14783,7,2544,2304,159570770829631962,344.957397,s3://viirs/PreProc/VIIRS_2020_95clear_192x192_...,1018284,25.739990,...,0,25.282654,24.600006,25.709991,25.600006,24.970001,1.0,0.630005,410.688721,-65.731323
733,5429,14783,7,2160,1440,160070760329631929,284.378540,s3://viirs/PreProc/VIIRS_2020_95clear_192x192_...,1099645,26.709991,...,0,26.213284,24.829987,26.929993,26.709991,25.529999,1.0,1.179993,313.803589,-29.425049


In [5]:
row_index = 5430
col_index = 14681
location_subset = table_all[(table_all.row == row_index) & (table_all.col == col_index)]

In [6]:
location_subset['datetime']

152261    2012-07-23 09:07:03
167359    2012-08-10 09:07:03
236658    2012-10-24 09:07:05
514190    2013-08-20 09:07:04
791224    2014-06-26 09:07:04
812020    2014-07-28 09:07:03
1178587   2015-10-06 09:07:04
1188660   2015-10-19 09:07:04
2018982   2018-04-18 09:07:06
2855443   2020-09-30 09:07:03
Name: datetime, dtype: datetime64[ns]

In [9]:
pandas.set_option('display.max_colwidth', None)
location_subset[['lat', 'lon', 'VIIRS_LL', 'LL', 'LL_diff','datetime','VIIRS_filename']]

,lat,lon,VIIRS_LL,LL,LL_diff,datetime,VIIRS_filename
152261,-19.252130,114.264404,443.155518,617.902161,-174.746643,2012-07-23 09:07:03,s3://viirs/data/2012/205/20120723171000-OSPO-L2P_GHRSST-SSTsubskin-VIIRS_NPP-ACSPO_V2.61-v02.0-fv01.0.nc
167359,-19.256676,114.263590,406.137939,481.129028,-74.991089,2012-08-10 09:07:03,s3://viirs/data/2012/223/20120810052000-OSPO-L2P_GHRSST-SSTsubskin-VIIRS_NPP-ACSPO_V2.61-v02.0-fv01.0.nc
236658,-19.259645,114.272250,492.862732,521.931396,-29.068665,2012-10-24 09:07:05,s3://viirs/data/2012/298/20121024052000-OSPO-L2P_GHRSST-SSTsubskin-VIIRS_NPP-ACSPO_V2.61-v02.0-fv01.0.nc
514190,-19.252542,114.267746,278.187134,366.600220,-88.413086,2013-08-20 09:07:04,s3://viirs/data/2013/232/20130820174000-OSPO-L2P_GHRSST-SSTsubskin-VIIRS_NPP-ACSPO_V2.61-v02.0-fv01.0.nc
791224,-19.254526,114.267975,603.196899,643.774719,-40.577820,2014-06-26 09:07:04,s3://viirs/data/2014/177/20140626174000-OSPO-L2P_GHRSST-SSTsubskin-VIIRS_NPP-ACSPO_V2.61-v02.0-fv01.0.nc
812020,-19.259470,114.264560,497.399719,489.379456,8.020264,2014-07-28 09:07:03,s3://viirs/data/2014/209/20140728174000-OSPO-L2P_GHRSST-SSTsubskin-VIIRS_NPP-ACSPO_V2.61-v02.0-fv01.0.nc
1178587,-19.251347,114.265800,270.922729,390.326172,-119.403442,2015-10-06 09:07:04,s3://viirs/data/2015/279/20151006053000-OSPO-L2P_GHRSST-SSTsubskin-VIIRS_NPP-ACSPO_V2.61-v02.0-fv01.0.nc
1188660,-19.246570,114.268250,275.682434,303.235962,-27.553528,2015-10-19 09:07:04,s3://viirs/data/2015/292/20151019062000-OSPO-L2P_GHRSST-SSTsubskin-VIIRS_NPP-ACSPO_V2.61-v02.0-fv01.0.nc
2018982,-19.257496,114.276970,356.412842,499.831482,-143.418640,2018-04-18 09:07:06,s3://viirs/data/2018/108/20180418062000-OSPO-L2P_GHRSST-SSTsubskin-VIIRS_NPP-ACSPO_V2.61-v02.0-fv01.0.nc
2855443,-19.243120,114.261810,165.375671,224.659058,-59.283386,2020-09-30 09:07:03,s3://viirs/data/2020/274/20200930062000-OSPO-L2P_GHRSST-SSTsubskin-VIIRS_NPP-ACSPO_V2.61-v02.0-fv01.0.nc
